Equal Error rate

In [3]:
import os
import numpy as np
import librosa
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from audiomentations import Compose, TimeStretch, PitchShift, AddGaussianNoise
from tensorflow.keras.regularizers import l2
import joblib

X_train_augmented = np.load('/data/common_source/datasets/datasets_asvc2019/mfcc/X_train_augmented.npy')
X_eval_augmented = np.load('/data/common_source/datasets/datasets_asvc2019/mfcc/X_eval_augmented.npy')
y_eval = np.load('/data/common_source/datasets/datasets_asvc2019/mfcc/y_eval.npy')

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

# Create a StandardScaler instance
scaler = StandardScaler()

# Reshape the 3D data to 2D
X_val_2d = X_eval_augmented.reshape(-1, X_eval_augmented.shape[-1])
X_train_2d = X_train_augmented.reshape(-1, X_train_augmented.shape[-1])

# Fit the scaler to your training data and transform it
X_train_normalized = scaler.fit_transform(X_train_2d)
# Transform the validation and test data using the same scaler
X_val_normalized = scaler.transform(X_val_2d)

# Reshape the normalized data back to 3D
X_val_normalized = X_val_normalized.reshape(X_eval_augmented.shape)

# Shuffle the validation data (if needed)
X_val_normalized, y_eval = shuffle(X_val_normalized, y_eval, random_state=42)

In [6]:
# Load the model from the file
model = joblib.load('jonat-model.joblib')

# Make predictions on the evaluation set
y_test_pred = model.predict(X_val_normalized)

2023-11-15 04:06:51.812919: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-15 04:06:51.813052: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-11-15 04:06:51.813134: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-11-15 04:06:51.813213: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2023-11-15 04:06:51.856001: W tensorflow/stream_executor/platform/default/dso_loader.cc:64

2227/2227 [==============================] - 18s 8ms/step


In [7]:
# Assuming y_test is the true class labels for your test set
target_scores = y_test_pred[y_eval == 1]  # Target scores for positive instances
nontarget_scores = y_test_pred[y_eval == 0]  # Non-target scores for negative instances

In [8]:
def compute_det_curve(target_scores, nontarget_scores):

    n_scores = target_scores.size + nontarget_scores.size
    all_scores = np.concatenate((target_scores, nontarget_scores))
    labels = np.concatenate((np.ones(target_scores.size), np.zeros(nontarget_scores.size)))

    # Sort labels based on scores
    indices = np.argsort(all_scores, kind='mergesort')
    labels = labels[indices]

    # Compute false rejection and false acceptance rates
    tar_trial_sums = np.cumsum(labels)
    nontarget_trial_sums = nontarget_scores.size - (np.arange(1, n_scores + 1) - tar_trial_sums)

    frr = np.concatenate((np.atleast_1d(0), tar_trial_sums / target_scores.size))  # false rejection rates
    far = np.concatenate((np.atleast_1d(1), nontarget_trial_sums / nontarget_scores.size))  # false acceptance rates
    thresholds = np.concatenate((np.atleast_1d(all_scores[indices[0]] - 0.001), all_scores[indices]))  # Thresholds are the sorted scores

    return frr, far, thresholds


def compute_eer(target_scores, nontarget_scores):
    """ Returns equal error rate (EER) and the corresponding threshold. """
    frr, far, thresholds = compute_det_curve(target_scores, nontarget_scores)
    abs_diffs = np.abs(frr - far)
    min_index = np.argmin(abs_diffs)
    eer = np.mean((frr[min_index], far[min_index]))
    return eer, thresholds[min_index]

In [9]:
eer, eer_threshold = compute_eer(target_scores, nontarget_scores)

print(f"Equal Error Rate (EER): {eer}")
print(f"Threshold at EER: {eer_threshold}")

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 3 dimension(s)